In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
df = pd.read_csv('all_players.csv')

In [3]:
df = df.dropna(subset=["Position", "Weak foot", "Skill moves"])

In [4]:
df_outfield = df[df['Position'] != 'GK'].copy()

In [5]:
attribute_weights = {
    'Striker': {
        'Finishing': 0.2, 'Shot Power': 0.15, 'Positioning': 0.15, 'Heading Accuracy': 0.1,
        'Sprint Speed': 0.1, 'Acceleration': 0.1, 'Strength': 0.1, 'Ball Control': 0.05, 'Dribbling': 0.05
    },
    'Winger': {
        'Dribbling': 0.2, 'Sprint Speed': 0.2, 'Acceleration': 0.15, 'Ball Control': 0.1,
        'Crossing': 0.1, 'Short Passing': 0.1, 'Finishing': 0.05, 'Curve': 0.05, 'Agility': 0.05
    },
    'Midfielder': {
        'Short Passing': 0.15, 'Long Passing': 0.15, 'Vision': 0.15, 'Ball Control': 0.1,
        'Dribbling': 0.1, 'Interceptions': 0.1, 'Stamina': 0.1, 'Standing Tackle': 0.05, 'Long Shots': 0.05
    },
    'Full-back': {
        'Sprint Speed': 0.15, 'Standing Tackle': 0.15, 'Sliding Tackle': 0.15, 'Crossing': 0.1,
        'Interceptions': 0.1, 'Def Awareness': 0.1, 'Stamina': 0.1, 'Acceleration': 0.05, 'Short Passing': 0.05
    },
    'Center-back': {
        'Def Awareness': 0.2, 'Standing Tackle': 0.2, 'Sliding Tackle': 0.15, 'Strength': 0.15,
        'Heading Accuracy': 0.1, 'Jumping': 0.1, 'Interceptions': 0.05, 'Aggression': 0.05
    }
}

In [6]:
for position, weights in attribute_weights.items():
    df_outfield[f'{position}_Score'] = sum(df_outfield[attr] * weight for attr, weight in weights.items())

In [7]:
outfield_features = [f'{pos}_Score' for pos in attribute_weights.keys()]
X_outfield = df_outfield[outfield_features]
y_outfield = df_outfield['Position']

# Encode the target variable
le_outfield = LabelEncoder()
y_outfield_encoded = le_outfield.fit_transform(y_outfield)

In [8]:
X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(
    X_outfield, y_outfield_encoded, test_size=0.2, random_state=42, stratify=y_outfield_encoded
)

In [9]:
scaler_out = StandardScaler()
X_train_out_scaled = scaler_out.fit_transform(X_train_out)
X_test_out_scaled = scaler_out.transform(X_test_out)

In [10]:
xgb_outfield = XGBClassifier(objective='multi:softmax', num_class=len(le_outfield.classes_), use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_outfield.fit(X_train_out_scaled, y_train_out)

E:\Data Science projects\Fifa dataset\fifa_project\Lib\site-packages\xgboost\training.py:199: UserWarning: [20:32:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'multi:softmax'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'mlogloss'


In [11]:
y_pred_out = xgb_outfield.predict(X_test_out_scaled)
print("Outfield Players Model Evaluation")
print(f"Accuracy: {accuracy_score(y_test_out, y_pred_out)}")
print(classification_report(y_test_out, y_pred_out, target_names=le_outfield.classes_))

Outfield Players Model Evaluation
Accuracy: 0.6086404066073697
              precision    recall  f1-score   support

         CAM       0.45      0.50      0.47       205
          CB       0.87      0.93      0.90       637
         CDM       0.58      0.55      0.57       289
          CM       0.62      0.72      0.67       428
          LB       0.43      0.39      0.41       267
          LM       0.24      0.23      0.24       207
          LW       0.10      0.05      0.07        77
          RB       0.42      0.41      0.42       281
          RM       0.26      0.22      0.24       192
          RW       0.16      0.09      0.11        80
          ST       0.85      0.89      0.87       485

    accuracy                           0.61      3148
   macro avg       0.45      0.45      0.45      3148
weighted avg       0.59      0.61      0.60      3148



In [12]:
def predict_multiple_positions(players):
    """
    Predicts the positions for a list of outfield players.

    Args:
    players (list of dict): A list where each dictionary contains the
                             attributes for a single player.

    Returns:
    list: A list of predicted position strings for each player.
    """
    # Create a DataFrame from the input list
    player_df = pd.DataFrame(players)

    # Calculate position-specific scores using the same weights as before
    for position, weights in attribute_weights.items():
        player_df[f'{position}_Score'] = sum(player_df[attr] * weight for attr, weight in weights.items())

    # Select the feature columns
    features = [f'{pos}_Score' for pos in attribute_weights.keys()]
    X_new = player_df[features]

    # Scale the new data using the previously fitted scaler
    X_new_scaled = scaler_out.transform(X_new)

    # Predict the encoded labels
    predictions_encoded = best_xgb_model.predict(X_new_scaled)

    # Decode the predictions to get the actual position names
    predictions = le_outfield.inverse_transform(predictions_encoded)

    return predictions


In [20]:
# Create a list of sample players with different skill sets
sample_players = [
    # Player 1: A classic striker
    {'Finishing': 95, 'Shot Power': 90, 'Positioning': 92, 'Heading Accuracy': 85, 'Sprint Speed': 88, 'Acceleration': 86, 'Strength': 80, 'Ball Control': 85, 'Dribbling': 82, 'Crossing': 70, 'Short Passing': 75, 'Curve': 70, 'Agility': 75, 'Long Passing': 60, 'Vision': 70, 'Interceptions': 40, 'Stamina': 80, 'Standing Tackle': 30, 'Long Shots': 80, 'Sliding Tackle': 25, 'Def Awareness': 35, 'Jumping': 85, 'Aggression': 70},
    # Player 2: A strong center-back
    {'Finishing': 40, 'Shot Power': 60, 'Positioning': 30, 'Heading Accuracy': 92, 'Sprint Speed': 70, 'Acceleration': 65, 'Strength': 95, 'Ball Control': 65, 'Dribbling': 55, 'Crossing': 50, 'Short Passing': 70, 'Curve': 40, 'Agility': 60, 'Long Passing': 75, 'Vision': 60, 'Interceptions': 90, 'Stamina': 85, 'Standing Tackle': 94, 'Long Shots': 50, 'Sliding Tackle': 91, 'Def Awareness': 93, 'Jumping': 90, 'Aggression': 88},
    # Player 3: A creative midfielder
    {'Finishing': 78, 'Shot Power': 82, 'Positioning': 85, 'Heading Accuracy': 60, 'Sprint Speed': 75, 'Acceleration': 80, 'Strength': 70, 'Ball Control': 92, 'Dribbling': 90, 'Crossing': 86, 'Short Passing': 94, 'Curve': 88, 'Agility': 85, 'Long Passing': 91, 'Vision': 95, 'Interceptions': 75, 'Stamina': 88, 'Standing Tackle': 70, 'Long Shots': 85, 'Sliding Tackle': 65, 'Def Awareness': 68, 'Jumping': 65, 'Aggression': 72},
]

# Get the predictions for the sample players
predicted_positions = predict_multiple_positions(sample_players)
print(f"Predicted positions: {predicted_positions}")

Predicted positions: ['ST' 'CB' 'CM']


### Gird Search

In [14]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}

In [15]:
xgb_grid = XGBClassifier(objective='multi:softmax',
                         num_class=len(le_outfield.classes_),
                         use_label_encoder=False,
                         eval_metric='mlogloss',
                         random_state=42)

In [17]:
grid_search = GridSearchCV(estimator=xgb_grid,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=3, # Using 5 folds for cross-validation
                           verbose=1,
                           n_jobs=-1) # Use all available CPU cores

# --- Fit the Grid Search to the training data ---
# This will take some time to run as it's training multiple models
grid_search.fit(X_train_out_scaled, y_train_out)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


E:\Data Science projects\Fifa dataset\fifa_project\Lib\site-packages\xgboost\training.py:199: UserWarning: [20:34:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,estimator,"XGBClassifier...class=11, ...)"
,param_grid,"{'learning_rate': [0.05, 0.1, ...], 'max_depth': [3, 5, ...], 'n_estimators': [100, 200, ...], 'subsample': [0.8, 1.0]}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,3
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,objective,'multi:softmax'


In [18]:
print(f"Best Parameters found: {grid_search.best_params_}")
print(f"Best Accuracy found: {grid_search.best_score_}")

Best Parameters found: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.8}
Best Accuracy found: 0.6174740548482927


In [19]:
best_xgb_model = grid_search.best_estimator_
y_pred_best = best_xgb_model.predict(X_test_out_scaled)

print("\n--- Evaluation with Best Model from Grid Search ---")
print(f"Accuracy: {accuracy_score(y_test_out, y_pred_best)}")
print(classification_report(y_test_out, y_pred_best, target_names=le_outfield.classes_))


--- Evaluation with Best Model from Grid Search ---
Accuracy: 0.6245235069885642
              precision    recall  f1-score   support

         CAM       0.46      0.52      0.49       205
          CB       0.88      0.95      0.91       637
         CDM       0.59      0.58      0.59       289
          CM       0.62      0.73      0.67       428
          LB       0.42      0.37      0.39       267
          LM       0.32      0.33      0.33       207
          LW       0.17      0.04      0.06        77
          RB       0.40      0.41      0.40       281
          RM       0.31      0.24      0.27       192
          RW       0.27      0.04      0.07        80
          ST       0.84      0.91      0.87       485

    accuracy                           0.62      3148
   macro avg       0.48      0.47      0.46      3148
weighted avg       0.60      0.62      0.61      3148



In [23]:
joblib.dump(best_xgb_model, "models/best_xgb_model.pkl")
print("✅ Model saved as best_xgb_model.pkl")

✅ Model saved as best_xgb_model.pkl


In [24]:
joblib.dump(le_outfield, 'models/position_label_encoder.pkl')

['models/position_label_encoder.pkl']

In [25]:
joblib.dump(scaler_out, 'models/position_feature_scaler.pkl')

['models/position_feature_scaler.pkl']

### Testing

In [44]:
def load_all_components():
    """
    Loads all the necessary models, scalers, and the encoder from disk.
    """
    components = {
        'position_model': joblib.load('models/best_xgb_model.pkl'),
        'position_encoder': joblib.load('models/position_label_encoder.pkl'),
        'position_scaler': joblib.load('models/position_feature_scaler.pkl'),
        'ovr_model': joblib.load('models/best_xgb_regr_oerall.pkl'),
        'ovr_scaler': joblib.load('models/ovr_feature_scaler.pkl'),
        'league_model': joblib.load('models/league_tier_model.pkl'),
        'league_encoder': joblib.load('models/league_tier_encoder.pkl'),
        # New components for similarity search
        'similarity_scaler': joblib.load('models/similarity_scaler.pkl'),
        'scaled_attributes': np.load('models/scaled_attributes.npy'),
        'player_info': pd.read_csv('models/player_info.csv')
        
    }
    return components

In [53]:
def predict_player_profile(player_attributes, num_similar=5):
    """
    Predicts the position, OVR, and league tier for a single player
    from their attributes.

    Args:
        player_attributes (dict): A dictionary containing the player's attributes.

    Returns:
        dict: A dictionary with 'predicted_position', 'predicted_ovr',
              and 'predicted_league_tier'.
    """
    # Load all required model components
    components = load_all_components()
    player_df = pd.DataFrame([player_attributes])

    # --- 1. OVR Prediction (Done first as it's a feature for the league model) ---
    ovr_attribute_columns = [
        'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Acceleration', 'Sprint Speed',
        'Positioning', 'Finishing', 'Shot Power', 'Long Shots', 'Volleys', 'Penalties',
        'Vision', 'Crossing', 'Free Kick Accuracy', 'Short Passing', 'Long Passing',
        'Curve', 'Dribbling', 'Agility', 'Balance', 'Reactions', 'Ball Control',
        'Composure', 'Interceptions', 'Heading Accuracy', 'Def Awareness',
        'Standing Tackle', 'Sliding Tackle', 'Jumping', 'Stamina', 'Strength', 'Aggression'
    ]
    X_ovr = player_df[ovr_attribute_columns]
    X_ovr_scaled = components['ovr_scaler'].transform(X_ovr)
    predicted_ovr = components['ovr_model'].predict(X_ovr_scaled)[0]
    predicted_ovr_rounded = round(float(predicted_ovr))

    # --- 2. Position Prediction ---
    attribute_weights = {
        'Striker': {'Finishing': 0.2, 'Shot Power': 0.15, 'Positioning': 0.15, 'Heading Accuracy': 0.1, 'Sprint Speed': 0.1, 'Acceleration': 0.1, 'Strength': 0.1, 'Ball Control': 0.05, 'Dribbling': 0.05},
        'Winger': {'Dribbling': 0.2, 'Sprint Speed': 0.2, 'Acceleration': 0.15, 'Ball Control': 0.1, 'Crossing': 0.1, 'Short Passing': 0.1, 'Finishing': 0.05, 'Curve': 0.05, 'Agility': 0.05},
        'Midfielder': {'Short Passing': 0.15, 'Long Passing': 0.15, 'Vision': 0.15, 'Ball Control': 0.1, 'Dribbling': 0.1, 'Interceptions': 0.1, 'Stamina': 0.1, 'Standing Tackle': 0.05, 'Long Shots': 0.05},
        'Full-back': {'Sprint Speed': 0.15, 'Standing Tackle': 0.15, 'Sliding Tackle': 0.15, 'Crossing': 0.1, 'Interceptions': 0.1, 'Def Awareness': 0.1, 'Stamina': 0.1, 'Acceleration': 0.05, 'Short Passing': 0.05},
        'Center-back': {'Def Awareness': 0.2, 'Standing Tackle': 0.2, 'Sliding Tackle': 0.15, 'Strength': 0.15, 'Heading Accuracy': 0.1, 'Jumping': 0.1, 'Interceptions': 0.05, 'Aggression': 0.05}
    }
    for position, weights in attribute_weights.items():
        player_df[f'{position}_Score'] = sum(player_df[attr] * weight for attr, weight in weights.items())
    position_features = [f'{pos}_Score' for pos in attribute_weights.keys()]
    X_pos_scaled = components['position_scaler'].transform(player_df[position_features])
    pos_pred_encoded = components['position_model'].predict(X_pos_scaled)
    predicted_position = components['position_encoder'].inverse_transform(pos_pred_encoded)[0]

    # --- 3. League Tier Prediction (FIXED SECTION) ---
    # Add the predicted OVR to the main player DataFrame
    predicted_league = ''
    if predicted_ovr_rounded >= 86:
        predicted_league = 'Top Tier'
    elif predicted_ovr_rounded < 72:
        predicted_league = 'Lower Tier'
    else:
        # Use the ML model only for the players in the middle range
        player_df['OVR'] = predicted_ovr_rounded
        league_feature_order = ['OVR', 'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY']
        X_league = player_df[league_feature_order]
        league_pred_encoded = components['league_model'].predict(X_league)
        predicted_league = components['league_encoder'].inverse_transform(league_pred_encoded)[0]
    similarity_columns = ['PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY', 'Acceleration', 'Sprint Speed','Positioning', 'Finishing', 'Shot Power', 'Long Shots', 'Volleys', 'Penalties','Vision', 'Crossing', 'Free Kick Accuracy', 'Short Passing', 'Long Passing','Curve', 'Dribbling', 'Agility', 'Balance', 'Reactions', 'Ball Control','Composure', 'Interceptions', 'Heading Accuracy', 'Def Awareness','Standing Tackle', 'Sliding Tackle', 'Jumping', 'Stamina', 'Strength', 'Aggression']
    input_player_scaled = components['similarity_scaler'].transform(player_df[similarity_columns])
    distances = euclidean_distances(input_player_scaled, components['scaled_attributes'])
    similar_player_indices = distances.argsort().flatten()[1 : num_similar + 1]
    similar_players_df = components['player_info'].iloc[similar_player_indices]
    similar_players = similar_players_df.to_dict('records')

    return {
        'predicted_position': predicted_position,
        'predicted_ovr': predicted_ovr_rounded,
        'predicted_league_tier': predicted_league,
        'similar_players': similar_players
    }

In [56]:
new_player_profile = {
    'Finishing': 80, 'Shot Power': 82, 'Positioning': 85, 'Heading Accuracy': 60,
    'Sprint Speed': 92, 'Acceleration': 95, 'Strength': 70, 'Ball Control': 88,
    'Dribbling': 91, 'Crossing': 84, 'Short Passing': 86, 'Curve': 85, 'Agility': 94,
    'Long Passing': 78, 'Vision': 85, 'Interceptions': 55, 'Stamina': 85,
    'Standing Tackle': 45, 'Sliding Tackle': 40, 'Def Awareness': 40, 'Jumping': 70,
    'Aggression': 65, 'PAC': 93, 'SHO': 84, 'PAS': 85, 'DRI': 90, 'DEF': 42,
    'PHY': 72, 'Long Shots': 80, 'Volleys': 75, 'Penalties': 79, 'Free Kick Accuracy': 70,
    'Balance': 90, 'Reactions': 88, 'Composure': 86
}

# Get the complete player profile prediction
full_profile = predict_player_profile(new_player_profile)

print("--- Full Player Profile Prediction ---")
print(f"  - Predicted Position: {full_profile['predicted_position']}")
print(f"  - Predicted Overall (OVR): {full_profile['predicted_ovr']}")
print(f"  - Predicted League Tier: {full_profile['predicted_league_tier']}")
print("\n--- Top 5 Similar Players ---")
for player in full_profile['similar_players']:
    print(f"  - {player['Name']} ({player['OVR']} {player['Position']}) - {player['Team']}")

--- Full Player Profile Prediction ---
  - Predicted Position: RW
  - Predicted Overall (OVR): 89
  - Predicted League Tier: Top Tier

--- Top 5 Similar Players ---
  - Mallory Swanson (87 LM) - Chicago Red Stars
  - Leroy Sané (85 RM) - FC Bayern München
  - Debinha (88 CAM) - KC Current
  - Federico Chiesa (84 RW) - Liverpool
  - Raphinha (84 RW) - FC Barcelona


In [57]:
# --- Test Player 2: World-Class Defender ---
elite_defender = {
    'Finishing': 45, 'Shot Power': 60, 'Positioning': 50, 'Heading Accuracy': 92,
    'Sprint Speed': 80, 'Acceleration': 75, 'Strength': 94, 'Ball Control': 72,
    'Dribbling': 65, 'Crossing': 58, 'Short Passing': 78, 'Curve': 55, 'Agility': 68,
    'Long Passing': 75, 'Vision': 65, 'Interceptions': 93, 'Stamina': 88,
    'Standing Tackle': 95, 'Sliding Tackle': 91, 'Def Awareness': 94, 'Jumping': 90,
    'Aggression': 89, 'PAC': 78, 'SHO': 55, 'PAS': 72, 'DRI': 68, 'DEF': 94,
    'PHY': 91, 'Long Shots': 50, 'Volleys': 45, 'Penalties': 60, 'Free Kick Accuracy': 50,
    'Balance': 60, 'Reactions': 88, 'Composure': 86
}

full_profile = predict_player_profile(elite_defender)
print("--- Full Player Profile Prediction ---")
print(f"  - Predicted Position: {full_profile['predicted_position']}")
print(f"  - Predicted Overall (OVR): {full_profile['predicted_ovr']}")
print(f"  - Predicted League Tier: {full_profile['predicted_league_tier']}")
print("\n--- Top 5 Similar Players ---")
for player in full_profile['similar_players']:
    print(f"  - {player['Name']} ({player['OVR']} {player['Position']}) - {player['Team']}")

--- Full Player Profile Prediction ---
  - Predicted Position: CB
  - Predicted Overall (OVR): 88
  - Predicted League Tier: Top Tier

--- Top 5 Similar Players ---
  - Éder Militão (85 CB) - Real Madrid
  - Cecilia Salvai (82 CB) - Juventus
  - Irene Paredes (88 CB) - FC Barcelona
  - Marquinhos (87 CB) - Paris SG
  - Virgil van Dijk (89 CB) - Liverpool


In [58]:
# --- Test Player 3: Young Prospect ---
young_prospect = {
    'Finishing': 64, 'Shot Power': 62, 'Positioning': 65, 'Heading Accuracy': 50,
    'Sprint Speed': 86, 'Acceleration': 88, 'Strength': 55, 'Ball Control': 74,
    'Dribbling': 76, 'Crossing': 65, 'Short Passing': 66, 'Curve': 68, 'Agility': 85,
    'Long Passing': 58, 'Vision': 64, 'Interceptions': 35, 'Stamina': 68,
    'Standing Tackle': 38, 'Sliding Tackle': 34, 'Def Awareness': 36, 'Jumping': 65,
    'Aggression': 50, 'PAC': 87, 'SHO': 63, 'PAS': 63, 'DRI': 77, 'DEF': 38,
    'PHY': 58, 'Long Shots': 60, 'Volleys': 55, 'Penalties': 62, 'Free Kick Accuracy': 54,
    'Balance': 88, 'Reactions': 69, 'Composure': 67
}

full_profile = predict_player_profile(young_prospect)
print("--- Full Player Profile Prediction ---")
print(f"  - Predicted Position: {full_profile['predicted_position']}")
print(f"  - Predicted Overall (OVR): {full_profile['predicted_ovr']}")
print(f"  - Predicted League Tier: {full_profile['predicted_league_tier']}")
print("\n--- Top 5 Similar Players ---")
for player in full_profile['similar_players']:
    print(f"  - {player['Name']} ({player['OVR']} {player['Position']}) - {player['Team']}")

--- Full Player Profile Prediction ---
  - Predicted Position: LM
  - Predicted Overall (OVR): 72
  - Predicted League Tier: Lower Tier

--- Top 5 Similar Players ---
  - Largie Ramazani (71 RM) - Leeds United
  - Yunus Akgün (72 RM) - Galatasaray
  - Elias Achouri (73 LW) - F.C. København
  - Anderson Oliveira (72 LM) - Suwon FC
  - Félix Correia (72 LM) - Gil Vicente


In [43]:
# --- Test Player 4: Solid Winger ---
solid_winger = {
    'Finishing': 74, 'Shot Power': 80, 'Positioning': 79, 'Heading Accuracy': 60,
    'Sprint Speed': 88, 'Acceleration': 87, 'Strength': 72, 'Ball Control': 80,
    'Dribbling': 81, 'Crossing': 80, 'Short Passing': 76, 'Curve': 75, 'Agility': 84,
    'Long Passing': 68, 'Vision': 77, 'Interceptions': 52, 'Stamina': 85,
    'Standing Tackle': 55, 'Sliding Tackle': 51, 'Def Awareness': 50, 'Jumping': 68,
    'Aggression': 74, 'PAC': 88, 'SHO': 76, 'PAS': 75, 'DRI': 81, 'DEF': 53,
    'PHY': 74, 'Long Shots': 75, 'Volleys': 68, 'Penalties': 72, 'Free Kick Accuracy': 65,
    'Balance': 86, 'Reactions': 79, 'Composure': 78
}

winger_prediction = predict_player_profile(solid_winger)
print("--- Solid Winger Prediction ---")
print(f"  - Predicted Position: {winger_prediction['predicted_position']}")
print(f"  - Predicted Overall (OVR): {winger_prediction['predicted_ovr']}")
print(f"  - Predicted League Tier: {winger_prediction['predicted_league_tier']}")
# Expected OVR: ~79, Position: RW, League: Mid Tier / Top Tier

--- Solid Winger Prediction ---
  - Predicted Position: LW
  - Predicted Overall (OVR): 80
  - Predicted League Tier: Lower Tier
